In [1]:
from __future__ import print_function

import sys
import datetime
from timeit import default_timer as timer
import numpy as np

import numba
from numba import cuda, jit

**Version information:**

In [2]:
print("This file is generated on:", datetime.datetime.now())
print("python: {0}.{1}".format(*sys.version_info[:2]))
print("numpy:", np.__version__)
print("numba:", numba.__version__)
print("CUDA GPU:", cuda.gpus[0].name)

This file is generated on: 2015-06-16 11:06:46.189817
python: 3.4
numpy: 1.9.2
numba: 0.19.1
CUDA GPU:

 b'GeForce GT 650M'


## NumPy CPU Jacobi Relaxation

In [3]:
def numpy_jacobi_core(A, Anew):
    error = 0.0
    n = A.shape[0]
    m = A.shape[1]

    for j in range(1, n - 1):
        for i in range(1, m - 1):
            Anew[j, i] = 0.25 * (A[j, i + 1] + A[j, i - 1]\
                                 + A[j - 1, i] + A[j + 1, i])
            error = max(error, abs(Anew[j, i] - A[j, i]))
    return error

The CPU driver

In [4]:
def cpu_driver(corefn, NN, NM):
    A = np.zeros((NN, NM), dtype=np.float32)
    Anew = np.zeros((NN, NM), dtype=np.float32)

    n = NN
    m = NM
    iter_max = 1000

    tol = 1.0e-6
    error = 1.0

    for j in range(n):
        A[j, 0] = 1.0
        Anew[j, 0] = 1.0

    print("Jacobi relaxation Calculation: {0} x {1} mesh".format(n, m))

    ts = timer()
    iter = 0

    while error > tol and iter < iter_max:
        error = corefn(A, Anew)

        # swap A and Anew
        tmp = A
        A = Anew
        Anew = tmp

        if iter % 100 == 0:
            print("{0:5}, {1:.6f} (elapsed: {2} s)".format(iter, error,
                                                           timer() - ts))

        iter += 1

    runtime = timer() - ts
    print(" total: {0} s".format(runtime))
    return runtime

Test NumPy Version

In [5]:
numpy_runtime = cpu_driver(numpy_jacobi_core, 64, 64)

Jacobi relaxation Calculation: 64 x 64 mesh
    0, 0.250000 (elapsed: 0.02075148100266233 s)
  100, 0.002397 (elapsed: 2.0976980850027758 s)


  200, 0.001200 (elapsed: 4.164207710004121 s)


  300, 0.000787 (elapsed: 6.272993409002083 s)


  400, 0.000572 (elapsed: 8.37755734899838 s)


  500, 0.000438 (elapsed: 10.486467000999255 s)


  600, 0.000347 (elapsed: 12.596951696003089 s)


  700, 0.000281 (elapsed: 14.715802538004937 s)


  800, 0.000232 (elapsed: 16.816559642000357 s)


  900, 0.000194 (elapsed: 18.883557389999623 s)


 total: 20.950265270999807 s

## Numba JIT version

Compile

In [6]:
numba_jacobi_core = jit(numpy_jacobi_core)

Test Numba Version

In [7]:
numba_runtime = cpu_driver(numba_jacobi_core, 64, 64)

Jacobi relaxation Calculation: 64 x 64 mesh
    0, 0.250000 (elapsed: 0.13911526599986246 s)


  100, 0.002397 (elapsed: 0.1407958409981802 s)
  200, 0.001200 (elapsed: 0.14206178199674468 s)
  300, 0.000787 (elapsed: 0.14331037599913543 s)
  400, 0.000572 (elapsed: 0.14453572099591838 s)
  500, 0.000438 (elapsed: 0.14592619799805107 s)
  600, 0.000347 (elapsed: 0.14720108999608783 s)
  700, 0.000281 (elapsed: 0.14856801399582764 s)
  800, 0.000232 (elapsed: 0.14987949100031983 s)
  900, 0.000194 (elapsed: 0.15119220099586528 s)
 total: 0.15244309199624695 s


Speedup: Numba vs NumPy

In [8]:
print("Speedup {0}x".format(numpy_runtime / numba_runtime))

Speedup 137.4300730630391x


## CUDA Version

In [9]:
tpb = 16


@cuda.jit
def cuda_jacobi_core(A, Anew, error):
    err_sm = cuda.shared.array((tpb, tpb), dtype=float32)

    ty = cuda.threadIdx.x
    tx = cuda.threadIdx.y
    bx = cuda.blockIdx.x
    by = cuda.blockIdx.y

    n = A.shape[0]
    m = A.shape[1]

    i, j = cuda.grid(2)

    err_sm[ty, tx] = 0
    if j >= 1 and j < n - 1 and i >= 1 and i < m - 1:
        Anew[j, i] = 0.25 * (A[j, i + 1] + A[j, i - 1]\
                             + A[j - 1, i] + A[j + 1, i])
        err_sm[ty, tx] = Anew[j, i] - A[j, i]

    cuda.syncthreads()

    # max-reduce err_sm vertically
    t = tpb // 2
    while t > 0:
        if ty < t:
            err_sm[ty, tx] = max(err_sm[ty, tx], err_sm[ty + t, tx])
        t //= 2
        cuda.syncthreads()

    # max-reduce err_sm horizontally
    t = tpb // 2
    while t > 0:
        if tx < t and ty == 0:
            err_sm[ty, tx] = max(err_sm[ty, tx], err_sm[ty, tx + t])
        t //= 2
        cuda.syncthreads()

    if tx == 0 and ty == 0:
        error[by, bx] = err_sm[0, 0]

A GPU driver

In [10]:
def gpu_driver(gpucorefn, NN, NM):
    A = np.zeros((NN, NM), dtype=np.float32)
    Anew = np.zeros((NN, NM), dtype=np.float32)

    n = NN
    m = NM
    iter_max = 1000

    tol = 1.0e-6
    error = 1.0

    for j in range(n):
        A[j, 0] = 1.0
        Anew[j, 0] = 1.0

    print("Jacobi relaxation Calculation: {0} x {1} mesh".format(n, m))

    ts = timer()
    iter = 0

    blockdim = (tpb, tpb)
    griddim = (NN // blockdim[0], NM // blockdim[1])

    error_grid = np.zeros(griddim, dtype=np.float32)

    stream = cuda.stream()

    dA = cuda.to_device(A, stream)  # to device and don't come back
    dAnew = cuda.to_device(Anew, stream)  # to device and don't come back
    derror_grid = cuda.to_device(error_grid, stream)

    while error > tol and iter < iter_max:
        gpucorefn[griddim, blockdim, stream](dA, dAnew, derror_grid)

        derror_grid.to_host(stream)

        # error_grid is available on host
        stream.synchronize()

        error = np.abs(error_grid).max()

        # swap dA and dAnew
        dA, dAnew = dAnew, dA

        if iter % 100 == 0:
            print("{0:5}, {1:.6f} (elapsed: {2} s)".format(iter, error,
                                                           timer() - ts))

        iter += 1

    runtime = timer() - ts
    print(" total: {0} s".format(runtime))
    return runtime

Test CUDA Version

In [11]:
gpu_driver(cuda_jacobi_core, 64, 64)

Jacobi relaxation Calculation: 64 x 64 mesh


TypingError: Failed at nopython (nopython frontend)
Untyped global name 'float32'
File "<ipython-input-9-a7dae457451e>", line 6

Testing on Large Mesh

In [12]:
NN = NM = 2048
print("Numba CPU:")
numba_runtime = cpu_driver(numba_jacobi_core, NN, NM)
print("Numba CUDA:")
cuda_runtime = gpu_driver(cuda_jacobi_core, NN, NM)

print("CUDA speedup: {0}x".format(numba_runtime / cuda_runtime))

Numba CPU:
Jacobi relaxation Calculation: 2048 x 2048 mesh
    0, 0.250000 (elapsed: 0.02220275999570731 s)
  100, 0.002397 (elapsed: 1.3460983109980589 s)


  200, 0.001204 (elapsed: 2.789101460999518 s)


  300, 0.000804 (elapsed: 4.332533403001435 s)


  400, 0.000603 (elapsed: 5.905186267998943 s)


  500, 0.000483 (elapsed: 7.542186214996036 s)

...


  800, 0.000302 (elapsed: 12.694606503995601 s)


  900, 0.000269 (elapsed: 14.472487016995728 s)


 total: 16.22451932900003 s


Numba CUDA:
Jacobi relaxation Calculation: 2048 x 2048 mesh


TypingError: Failed at nopython (nopython frontend)
Untyped global name 'float32'
File "<ipython-input-9-a7dae457451e>", line 6